In [96]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

In [97]:
birdy_uri = 'spptify:artist:2WX2uTcsvV5OnS0inACecP'

results = sp.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

album_ids = []
for album in albums:
    album_ids.append(album['id'])

track_ids = []
for album in album_ids:

    results = sp.album_tracks(album)
    tracks = results['items']

    for track in tracks:
        track_ids.append(track['id'])

track_ids

['2mpezzl8FlxttNlaUSDzuV',
 '6VLEEyg7auxTMixCT8U3Gk',
 '0HtdR1K0cptJbOF3EKlAbT',
 '3QorTmBB8lYdSO9bGglOrb',
 '53vcgIyHUsxpeZjcEymEmh',
 '0TVYFZCASBjQrrPWBdyWHY',
 '3L5Vn1Mxq8x0EESC2BleaC',
 '5HsW6nVD3RJ4K7su9HyBGW',
 '5x8apEOj9Q0oxCmjIPK6qt',
 '31iO9bmNknTGoYvXkHjxPe',
 '0gnQNex28Am5f0TvQfnXnJ',
 '2hXwajVokY95lCzkAA2BaT',
 '0zgGH4S4DyxYDWf674Sy3M',
 '5B6lHw2K2P5SGUeblLdKHW',
 '6Y0LZeTw2GHLgbOsql7iMU',
 '0XWRZ1NBx1VzIrYUaGhga5',
 '0ugjtsLHei2JESMrmlJF7a',
 '0jCgiC6yOn6Vj9nGzOwsQ3',
 '3UbCMiXR6d9zcfNRS0mTzG',
 '555CH6CnGJRqO37pqA30vW',
 '6IEc8l3p6Z5OBLMFg6Z2V1',
 '1aFMbEwvxn0Qjqg5VsjlHv',
 '1z6ytxdIawe7Dd14cfiSQf',
 '2DjaAYJgTSIsFSZGbC7p7b',
 '57l9l0BtYWoFCDyZE1ug6Z',
 '7ltRjbYHE8G2HancQI67oN',
 '5jpIP0be2NytXkKhjCFV3p',
 '67m00DhUHLDonJoqu1fOHx',
 '6tljpHxh2KUFUbbpyUbmvw',
 '2lqMhO4DsPlchv63NHhbhg',
 '7LPdainM5tT2AseNSdXh4X',
 '5BYzhjvCKzFojV5Ka2e8G9',
 '6lyjWvSUgYtX26zfrQ6gn8',
 '4c9UU12Z3KZiC0K7696rup',
 '2iuL3ONbUGaLz5gmzk0PKo',
 '5sjPJRSH87lEMd1rkMn1Hl',
 '02Xxh1cWqOjSson8fwk7qJ',
 

In [100]:
# Create DataFrames of features from track ids
features = []
for track in track_ids:
    results = sp.audio_features(track)
    features.append(results[0])

data = pd.DataFrame.from_records(features)
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.364,0.0454,5,-26.645,1,0.0437,0.958,0.985000,0.1220,0.0528,111.013,audio_features,2mpezzl8FlxttNlaUSDzuV,spotify:track:2mpezzl8FlxttNlaUSDzuV,https://api.spotify.com/v1/tracks/2mpezzl8Flxt...,https://api.spotify.com/v1/audio-analysis/2mpe...,49507,5
1,0.349,0.4720,5,-12.792,1,0.0485,0.694,0.005680,0.1550,0.4610,88.167,audio_features,6VLEEyg7auxTMixCT8U3Gk,spotify:track:6VLEEyg7auxTMixCT8U3Gk,https://api.spotify.com/v1/tracks/6VLEEyg7auxT...,https://api.spotify.com/v1/audio-analysis/6VLE...,266480,4
2,0.595,0.5600,1,-7.988,0,0.0327,0.536,0.000000,0.1560,0.4830,131.833,audio_features,0HtdR1K0cptJbOF3EKlAbT,spotify:track:0HtdR1K0cptJbOF3EKlAbT,https://api.spotify.com/v1/tracks/0HtdR1K0cptJ...,https://api.spotify.com/v1/audio-analysis/0Htd...,202400,4
3,0.577,0.4150,9,-10.550,1,0.0514,0.825,0.000002,0.0887,0.4630,91.949,audio_features,3QorTmBB8lYdSO9bGglOrb,spotify:track:3QorTmBB8lYdSO9bGglOrb,https://api.spotify.com/v1/tracks/3QorTmBB8lYd...,https://api.spotify.com/v1/audio-analysis/3Qor...,206840,4
4,0.353,0.3860,0,-10.927,1,0.0450,0.661,0.000231,0.0806,0.2690,147.808,audio_features,53vcgIyHUsxpeZjcEymEmh,spotify:track:53vcgIyHUsxpeZjcEymEmh,https://api.spotify.com/v1/tracks/53vcgIyHUsxp...,https://api.spotify.com/v1/audio-analysis/53vc...,234240,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,0.316,0.2760,7,-8.902,1,0.0347,0.944,0.000712,0.3670,0.2340,96.165,audio_features,66p38eJ5BjCBjjM61fidvH,spotify:track:66p38eJ5BjCBjjM61fidvH,https://api.spotify.com/v1/tracks/66p38eJ5BjCB...,https://api.spotify.com/v1/audio-analysis/66p3...,286773,3
219,0.460,0.4510,2,-6.800,1,0.0252,0.497,0.000005,0.1550,0.1440,77.928,audio_features,2pxu6XNg4oW5MOWw19lhTq,spotify:track:2pxu6XNg4oW5MOWw19lhTq,https://api.spotify.com/v1/tracks/2pxu6XNg4oW5...,https://api.spotify.com/v1/audio-analysis/2pxu...,283880,4
220,0.347,0.4310,0,-7.692,1,0.0271,0.427,0.012500,0.0586,0.2080,79.951,audio_features,4ZXa883a23f8Ei1Cg0zt5K,spotify:track:4ZXa883a23f8Ei1Cg0zt5K,https://api.spotify.com/v1/tracks/4ZXa883a23f8...,https://api.spotify.com/v1/audio-analysis/4ZXa...,537360,4
221,0.391,0.0849,11,-13.502,1,0.0436,0.926,0.000010,0.0898,0.2860,66.542,audio_features,08pzZzYbMd1TYVJRzeK0FT,spotify:track:08pzZzYbMd1TYVJRzeK0FT,https://api.spotify.com/v1/tracks/08pzZzYbMd1T...,https://api.spotify.com/v1/audio-analysis/08pz...,122667,5


In [99]:
# Save data to csv in subfolder
data.to_csv('./data/test_data.csv', encoding='utf-8')

In [101]:
# Live
sp.audio_features('2RpXUwCytDFqZSHl6kIquB')

[{'danceability': 0.26,
  'energy': 0.99,
  'key': 7,
  'loudness': -3.498,
  'mode': 1,
  'speechiness': 0.128,
  'acousticness': 0.00202,
  'instrumentalness': 0.00416,
  'liveness': 0.779,
  'valence': 0.0885,
  'tempo': 128.29,
  'type': 'audio_features',
  'id': '2RpXUwCytDFqZSHl6kIquB',
  'uri': 'spotify:track:2RpXUwCytDFqZSHl6kIquB',
  'track_href': 'https://api.spotify.com/v1/tracks/2RpXUwCytDFqZSHl6kIquB',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2RpXUwCytDFqZSHl6kIquB',
  'duration_ms': 349733,
  'time_signature': 4}]

In [103]:
# Remastered
sp.audio_features('0xzhryP1AoHUazYdJ5rj3B')

[{'danceability': 0.46,
  'energy': 0.922,
  'key': 2,
  'loudness': -7.388,
  'mode': 1,
  'speechiness': 0.264,
  'acousticness': 0.0449,
  'instrumentalness': 0.000252,
  'liveness': 0.34,
  'valence': 0.328,
  'tempo': 119.961,
  'type': 'audio_features',
  'id': '0xzhryP1AoHUazYdJ5rj3B',
  'uri': 'spotify:track:0xzhryP1AoHUazYdJ5rj3B',
  'track_href': 'https://api.spotify.com/v1/tracks/0xzhryP1AoHUazYdJ5rj3B',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0xzhryP1AoHUazYdJ5rj3B',
  'duration_ms': 311027,
  'time_signature': 4}]

In [104]:
# Non-remastered (original)
sp.audio_features('4BB3Itr3YETOKFURnC28Ld')

[{'danceability': 0.472,
  'energy': 0.859,
  'key': 2,
  'loudness': -11.866,
  'mode': 1,
  'speechiness': 0.181,
  'acousticness': 0.0793,
  'instrumentalness': 0.000202,
  'liveness': 0.645,
  'valence': 0.322,
  'tempo': 119.989,
  'type': 'audio_features',
  'id': '4BB3Itr3YETOKFURnC28Ld',
  'uri': 'spotify:track:4BB3Itr3YETOKFURnC28Ld',
  'track_href': 'https://api.spotify.com/v1/tracks/4BB3Itr3YETOKFURnC28Ld',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4BB3Itr3YETOKFURnC28Ld',
  'duration_ms': 310667,
  'time_signature': 4}]